# Audio Playback and Acceleration Signal

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette = palette2
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

In [ ]:
def process_df_for_zure_test(df):
    df.insert(len(df.columns), 'speaker_on_ffill', df['speaker_on'].ffill(inplace=False))
    df.insert(len(df.columns), 'play_audio_zero_fill', df['play_audio'].fillna(0))
    df.insert(len(df.columns), '_index', np.arange(0, len(df), 1))
    return df

def detect_acceleration_sensor_data_response(df, threshold=0.2):
    '''
    Very simple change point detection algorithm using a threshold value
    '''
    acc_x = df['acc_x'].values
    acc_y = df['acc_y'].values
    acc_z = df['acc_z'].values
    for i in range(1, len(df)+1, 1):
        acc_x_diff = np.abs(acc_x[i] - acc_x[i-1])
        acc_y_diff = np.abs(acc_y[i] - acc_y[i-1])
        acc_z_diff = np.abs(acc_z[i] - acc_z[i-1])
        total_abs_diff = acc_x_diff + acc_y_diff + acc_z_diff
        if total_abs_diff > threshold:
            return df.index[i], i # index in the df, and i
    return 0, 0


def plot_acc_and_speaker_data_for_zure_test(df, idx_from, idx_to, detection=False):
    df = df[idx_from:idx_to]
    fig, ax = plt.subplots(1, 1, figsize=(20, 6))
    ax = sns.lineplot(ax=ax, x='_index', y='acc_x', data=df, color=palette[0], label="x")
    ax = sns.lineplot(ax=ax, x='_index', y='acc_y', data=df, color=palette[2], label="y")
    ax = sns.lineplot(ax=ax, x='_index', y='acc_z', data=df, color=palette[1], label="z")
    ax.set_ylabel('Acc (g)')
    ax.set_ylim(-2.2, 2.2)
    ax.grid(which='major')
    ax.legend(ncol=3, loc="lower right")
    change_indices = df.index[(df['speaker_on_ffill'].shift(1) == 0) & (df['speaker_on_ffill'] == 1)].tolist()
    for idx in change_indices:
        ax.axvline(x=idx, color='black', linestyle='--')
        ax.axvspan(xmin=idx, xmax=idx+25*4.2, color='grey', alpha=0.3)
        ax.axvline(x=idx+3, color='black', linestyle='-') # shift 120 ms (40 ms * 3 samples) 
        ax.axvspan(xmin=idx+3, xmax=idx+3+25*4.2, color='grey', alpha=0.3)
    ax.set_xlim(idx_from, idx_to)
    ax.set_xlabel('t (data point)')
    ax.set_ylabel('Acc (g)')

    if detection == True:
        THRESHOLD = 0.2
        response_idx, _i = detect_acceleration_sensor_data_response(df, threshold=THRESHOLD)
        ax.axvline(x=response_idx, color='green', linestyle='-')
    plt.show()
    # plt.close()
    return fig, df, change_indices

## Path

In [ ]:
# Change the path appropriately
# base_dir = "../path_to_base_dir"
base_dir = "C:/Users/ryoma/D/logbot-data/umineko/Umineko2024/v5-umineko-2024-playback/v5-spk-acc-zure-test/experiment-data"

input_path = f"{base_dir}/device-03/trial-01/logdata/logdata.csv" # 11 trials
path1 = input_path

input_path = f"{base_dir}/device-04/trial-01/logdata/logdata.csv" # 8 trials
path2 = input_path
path_list = [path1, path2]

## Run calculation

In [ ]:
OFFSET_VALUE_MS = 338
device_list = []
trial_list = []
playback_count_list = []
idx_diff_list = []
time_diff_ms_list = []
time_delay_ms_list = []
for p, path in enumerate(path_list):
    print(f"input_path: {path}")
    device = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(path))))
    trial = os.path.basename(os.path.dirname(os.path.dirname(path)))
    df = pd.read_csv(path)
    df = process_df_for_zure_test(df)
    fig, _df, playback_indices = plot_acc_and_speaker_data_for_zure_test(df, 0, len(df))
    print(f"playback_indices: {playback_indices}")

    for i in range(0, len(playback_indices), 1):
        idx = playback_indices[i]
        fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=idx-1, idx_to=idx+30+1, detection=True)
        response_idx, _i = detect_acceleration_sensor_data_response(_df, threshold=0.2)
        idx_diff = response_idx - idx
        time_diff_ms = idx_diff * 1000 / 25
        print(f"playback: {i+1}")
        print(f"idx = {idx} | response_idx: {response_idx} | i: {_i}")
        print(f"idx_diff = {idx_diff} | diff (ms): {time_diff_ms}")

        device_list.append(device)
        trial_list.append(trial)
        playback_count_list.append(i)
        idx_diff_list.append(idx_diff)
        time_diff_ms_list.append(time_diff_ms)
        time_delay_ms_list.append(time_diff_ms - OFFSET_VALUE_MS)

data_dict = {
    'device': device_list,
    'trial': trial_list,
    'playback_count': playback_count_list,
    'idx_diff': idx_diff_list,
    'time_diff': time_diff_ms_list, # ms
    'time_delay': time_delay_ms_list # ms
}
df_results = pd.DataFrame(data_dict)

# black lines: playback timing
# green lines: detected change points

## Fig. S05

In [ ]:
np.random.seed(558)

df = df_results
# y_colname = 'time_diff'
y_colname = 'time_delay'
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

median = np.median(df[y_colname])
min = np.min(df[y_colname])
max = np.max(df[y_colname])

ax.axhline(xmin=0, xmax=10, y=max, color=palette[0], linestyle='-',    label=f'max      = {max:.2f}')
ax.axhline(xmin=0, xmax=10, y=median, color=palette[1], linestyle="-", label=f'median = {median:.2f}')
ax.axhline(xmin=0, xmax=10, y=min, color=palette[2], linestyle='-',    label=f'min       = {min:.2f}')

ax.yaxis.set_minor_locator(tck.MultipleLocator(50))
ax.grid(which='minor', axis='y', linestyle='-', linewidth='0.75')
ax.grid(which='major', axis='y', linestyle='-', linewidth='0.75')

# Violin plot
violin_parts = sns.violinplot(
    x='device', y=y_colname, data=df, 
    inner=None,
    linewidth=0.0, color='#333333', saturation=0.5,
    ax=ax
)
for pc in violin_parts.collections:
    pc.set_edgecolor('black')
    pc.set_facecolor('#333333')
    pc.set_alpha(0.25)

# Box plot
unique_devices = df['device'].unique()
device_positions = np.arange(len(unique_devices)) + 0.25  # shift 0.25 to right
data_by_device = [df[df['device'] == device][y_colname] for device in unique_devices]

box_parts = ax.boxplot(
    data_by_device,
    positions=device_positions,
    widths=0.15,
    patch_artist=True,
    medianprops=dict(color='black', linewidth=3.0),
    whiskerprops=dict(color='black', linewidth=2.0),
    capprops=dict(color='black', linewidth=2.0),
    boxprops=dict(facecolor='white', color='black', linewidth=2.0),
)

ax.set_xticks(np.arange(len(unique_devices)))
ax.set_xticklabels(unique_devices)

np.random.seed(558)
sns.stripplot(x='device', y=y_colname, data=df, color="#333333", jitter=True, size=9, alpha=0.7, ax=ax)
ax.set_yticks(np.arange(0, 1300, 100))

if y_colname == 'time_diff':
    ax.set_ylim(410, 1240)
elif y_colname == 'time_delay':
    ax.set_ylim(110, 890)

plt.legend(ncol=1)
plt.xlabel('Device', labelpad=10)
plt.ylabel('Time difference (ms)', labelpad=10)
plt.show()
save_dir = "../output/figure-for-paper/"
# fig.savefig(f"{save_dir}/png/fig_s05_zure_spk_command_audio_time_diff.png", dpi=350, bbox_inches="tight", pad_inches=0.25, transparent=False)
# fig.savefig(f"{save_dir}/pdf/fig_s05_zure_spk_command_audio_time_diff.pdf", dpi=600, bbox_inches="tight", pad_inches=0.25, transparent=False)

## Test Program

In [ ]:
df = pd.read_csv(input_path)
df = process_df_for_zure_test(df)
fig, _df, playback_indices = plot_acc_and_speaker_data_for_zure_test(df, 0, len(df))
print(f"playback_indices: {playback_indices}")

In [ ]:
fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=1500, idx_to=3000)

In [ ]:
# Let's check the first trial
# the index range should cover the entire first trial
fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=1475, idx_to=2200) 
# print(_df.columns)
show_columns = [
    'rtc_year', 'rtc_month', 'rtc_day', 'rtc_hour', 'rtc_min', 'rtc_sec', 'rtc_msec',
    'camera_command', 'camera_recording', 'camera_count', 'play_audio', 'speaker_on', 'audio_file']
df_show = _df[_df['rtc_msec'] == 0][show_columns]
print(f"camera_command: {np.sum(df_show['camera_command'])}")     # recording time +1 (sec)
print(f"camera_recording: {np.sum(df_show['camera_recording'])}") # recording time (sec)
display(df_show)

In [ ]:
# Let's take a closer look
fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=1775, idx_to=1950)

In [ ]:
# Much closer look
fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=1800-1, idx_to=1830+1)
# fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=1800-1, idx_to=1830+1, detection=True)
# -> in this trial, the acceleration signal changed around the index 1816

In [ ]:
# Apply the change point detection algorithm to the first trial
print(playback_indices)
for i in range(0, len(playback_indices[:3]), 1):
    idx = playback_indices[i]
    fig, _df, _ = plot_acc_and_speaker_data_for_zure_test(df, idx_from=idx-1, idx_to=idx+30+1, detection=True)
    response_idx, _i = detect_acceleration_sensor_data_response(_df, threshold=0.2)
    idx_diff = response_idx - idx
    print(f"playback: {i+1}")
    print(f"idx = {idx} | response_idx: {response_idx} | i: {_i}")
    print(f"idx_diff = {idx_diff} | diff (ms): {idx_diff * 40}")


In [ ]:
display(_df[['_index', 'acc_x', 'acc_y', 'acc_z']].head(18))

In [ ]:
# Let's check how different threshold values influence the results
threshold_list = [0.1, 0.2, 0.3, 0.5, 1.0]
for t, threshold in enumerate(threshold_list):
    response_idx, _i = detect_acceleration_sensor_data_response(_df, threshold=threshold)
    print(f"threshold = {threshold} | response_idx: {response_idx} | i: {_i}")

In [ ]:
def calc_delay_time_ms(delay_sample_count, t_command, t_humna_aud_res, t_acc_res):
    t_total_response = delay_sample_count / 25 * 1000 # 25 Hz
    t_delay_time_ms = t_total_response - t_command - t_humna_aud_res - t_acc_res
    return t_delay_time_ms

In [ ]:
t_delay_time_ms = calc_delay_time_ms(16, 120, 228, 0)
print(f"{t_delay_time_ms} ms")

In [ ]:
print(f"{1000 * 16/25} ms")